In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# Check other null data points
conn = sqlite3.connect('./data/cars.db')
sql_query = "SELECT * FROM inventory"
#sql_query = "SELECT * FROM inventory_staging"
result = pd.read_sql_query(sql_query, conn)
result

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2009 Toyota Camry LE Sedan,2009,Toyota,Camry LE,Sedan,None,None,124351.0,8495.0,Magnetic Gray Metallic,Ash,Automatic,2.4L I4 158hp 161ft. lbs.,None,4T4BE46K89R114118,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
1,2011 BMW X3 xDrive28i SUV,2011,BMW,X3 xDrive28i,SUV,None,None,98952.0,13995.0,Jet Black,Black Nevada Leather,Automatic,3.0L I6 240hp 221ft. lbs.,None,5UXWX5C51BL715699,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
2,2011 Mercedes-Benz ML 350 4MATIC SUV,2011,Mercedes-Benz,ML 350 4MATIC,SUV,None,None,98922.0,14995.0,Obsidian Black Metallic,Black Leather,Automatic,3.5L V6 268hp 258ft. lbs.,None,4JGBB8GB4BA723044,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
3,2012 Audi A6 3.0T quattro Prestige Sedan,2012,Audi,A6 3.0T quattro Prestige,Sedan,None,None,98933.0,17995.0,White,Brown,Automatic,3.0L Supercharged V6 310hp 325ft. lbs.,None,WAUHGAFC4CN137257,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
4,2014 BMW 535i xDrive Sedan,2014,BMW,535i xDrive,Sedan,None,None,72940.0,23995.0,Silver,Tan,Automatic,3.0L Turbo I6 300hp 300ft. lbs.,None,WBA5B3C5XED537237,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7794,2011 Subaru Forester 2.5X Premium,2011,Subaru,Forester 2.5X Premium,Sport Utility,Forester,2.5X Premium,134757.0,9461.0,Sage Green Metallic,Platinum,Automatic,2.5L H4 16V MPFI DOHC,AWD,JF2SHADC9BH768545,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-07-07 21:38:28.699766
7795,2009 Jeep Patriot Sport,2009,Jeep,Patriot Sport,Sport Utility,Patriot,Sport,124129.0,0.0,Deep Water Blue Pearl,Dark Slate Gray,Variable,2.4L I4 16V MPFI DOHC,4WD,1J4FF28B29D121462,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-07-07 21:38:28.699766
7796,2009 Toyota Camry,2009,Toyota,Camry Base,4dr Car,Camry,Base,69793.0,10253.0,Black,,5-Speed,2.4L I4 16V MPFI DOHC,FWD,4T4BE46K79R086666,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-07-07 21:38:28.699766
7797,2007 Ford Edge SE,2007,Ford,Edge SE,4dr Car,Edge,SE,161003.0,0.0,White,,Automatic,3.5L V6 24V MPFI DOHC,AWD,2FMDK46C87BA30534,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-07-07 21:38:28.699766


In [4]:
result.loc[result['vin'].isnull(), :]

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
586,2018 Audi A4 2.0T quattro Premium Plus,2018,Audi,A4 2.0T quattro Premium Plus,None,None,None,15679.0,36590.0,Moonlight Blue Metallic,Nougat Brown Interior,Automatic 7-Speed,2.0L I4 Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
587,2017 Audi TT 2.0T quattro,2017,Audi,TT 2.0T quattro,None,None,None,29520.0,39480.0,Nano Gray Metallic,"Rock Gray, Fine Nappa Leather",Automatic 6-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
588,2018 Audi Q7 3.0T quattro Prestige,2018,Audi,Q7 3.0T quattro Prestige,None,None,None,53912.0,48560.0,Black,Brown,Automatic 8-Speed,V6 3.0L Supercharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
589,2017 Audi A4 2.0T quattro Premium Plus,2017,Audi,A4 2.0T quattro Premium Plus,None,None,None,53298.0,29350.0,White,Beige,Automatic 7-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
590,2018 Audi SQ5 3.0T quattro Prestige,2018,Audi,SQ5 3.0T quattro Prestige,None,None,None,99106.0,36850.0,Black,Black,Automatic 8-Speed,V6 3.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
591,2013 Audi A4 2.0T quattro Premium,2013,Audi,A4 2.0T quattro Premium,None,None,None,54283.0,17980.0,White,Black,Automatic 8-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
592,2018 Audi S4 3.0T quattro Prestige,2018,Audi,S4 3.0T quattro Prestige,None,None,None,62800.0,41580.0,Black,Black,Automatic 8-Speed,V6 3.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
593,2020 Audi Q5 2.0T quattro Premium Plus,2020,Audi,Q5 2.0T quattro Premium Plus,None,None,None,29709.0,47480.0,Mythos Black Metallic,"Nougat Brown, Leather Seating",Automatic 7-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
594,2018 Audi Q7 3.0T quattro Premium Plus,2018,Audi,Q7 3.0T quattro Premium Plus,None,None,None,99848.0,34980.0,Graphite Gray Metallic,Black,Automatic 8-Speed,V6 3.0L Supercharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708
595,2018 Audi S7 4.0T quattro Premium Plus,2018,Audi,S7 4.0T quattro Premium Plus,None,None,None,85521.0,48760.0,Mythos Black Metallic,Black,Automatic 7-Speed,V8 4.0L Twin Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-15 18:01:28.249708


In [ ]:
result.loc[result['vin'].isnull(), 'vin']